In [ ]:
import os
print os.path.join(caffe_root + 'models/vgg_face_caffe')
if os.path.isfile(caffe_root + 'models/vgg_face_caffe/VGG_FACE.caffemodel'):
    print 'Vgg Face Caffe found.'
else:
    print 'Downloading pre-trained Vgg Face Caffe model...'
    #!./scripts/download_model_from_gist.sh <gist_id> 
    !../scripts/download_model_binary.py ../models/bvlc_reference_caffenet

In [ ]:
##################################################################
# Subssidiary functions #
##################################################################

def make_sure_path_exists(directory):
    if not os.path.exists(directory):
        try:
            os.makedirs(directory)
        except OSError as error:
            if error.errno != errno.EEXIST:
                raise

In [ ]:
#Fuera de uso###################

from caffe import layers as L
from caffe import params as P

weight_param = dict(lr_mult=1, decay_mult=1)
bias_param   = dict(lr_mult=2, decay_mult=0)
learned_param = [weight_param, bias_param]

frozen_param = [dict(lr_mult=0)] * 2

def conv_relu(bottom, ks, nout, stride=1, pad=0, group=1,
              param=learned_param,
              weight_filler=dict(type='gaussian', std=0.01),
              bias_filler=dict(type='constant', value=0.1)):
    conv = L.Convolution(bottom, kernel_size=ks, stride=stride,
                         num_output=nout, pad=pad, group=group,
                         param=param, weight_filler=weight_filler,
                         bias_filler=bias_filler)
    return conv, L.ReLU(conv, in_place=True)

def fc_relu(bottom, nout, param=learned_param,
            weight_filler=dict(type='gaussian', std=0.005),
            bias_filler=dict(type='constant', value=0.1)):
    fc = L.InnerProduct(bottom, num_output=nout, param=param,
                        weight_filler=weight_filler,
                        bias_filler=bias_filler)
    return fc, L.ReLU(fc, in_place=True)

def max_pool(bottom, ks, stride=1):
    return L.Pooling(bottom, pool=P.Pooling.MAX, kernel_size=ks, stride=stride)

def vgg_face_net(data, label=None, train=True, num_classes=10,
             classifier_name='fc9_elq', learn_all=False):
    """Returns a NetSpec specifying vgg_face_net, following the original proto text
       specification (./M9/Methods/vgg_face_caffe/VGG_FACE_train_val.prototxt)."""
    n = caffe.NetSpec()
    n.data = data
    param = learned_param if learn_all else frozen_param
    n.conv1_1, n.relu1_1 = conv_relu(n.data, 3, 64, pad=1, param=learned_param)
    n.conv1_2, n.relu1_2 = conv_relu(n.relu1_1, 3, 64, pad=1, param=learned_param)
    n.pool1 = max_pool(n.relu1_2, 2, stride=2)
    #n.norm1 = L.LRN(n.pool1, local_size=5, alpha=1e-4, beta=0.75)
    n.conv2_1, n.relu2_1 = conv_relu(n.pool1, 3, 128, pad=1, param=learned_param)
    n.conv2_2, n.relu2_2 = conv_relu(n.relu2_1, 3, 128, pad=1, param=learned_param)
    n.pool2 = max_pool(n.relu2_2, 2, stride=2)
    #n.norm2 = L.LRN(n.pool2, local_size=5, alpha=1e-4, beta=0.75)
    n.conv3_1, n.relu3_1 = conv_relu(n.pool2, 3, 256, pad=1, param=learned_param)
    n.conv3_2, n.relu3_2 = conv_relu(n.relu3_1, 3, 256, pad=1, param=learned_param)
    n.conv3_3, n.relu3_3 = conv_relu(n.relu3_2, 3, 256, pad=1, param=learned_param)
    n.pool3 = max_pool(n.relu3_3, 2, stride=2)
    n.conv4_1, n.relu4_1 = conv_relu(n.pool3, 3, 512, pad=1, param=learned_param)
    n.conv4_2, n.relu4_2 = conv_relu(n.relu4_1, 3, 512, pad=1, param=learned_param)
    n.conv4_3, n.relu4_3 = conv_relu(n.relu4_2, 3, 512, pad=1, param=learned_param)
    n.pool4 = max_pool(n.relu4_3, 2, stride=2)
    n.conv5_1, n.relu5_1 = conv_relu(n.pool4, 3, 512, pad=1, param=learned_param)
    n.conv5_2, n.relu5_2 = conv_relu(n.relu5_1, 3, 512, pad=1, param=learned_param)
    n.conv5_3, n.relu5_3 = conv_relu(n.relu5_2, 3, 512, pad=1, param=learned_param)
    n.pool5 = max_pool(n.relu5_3, 2, stride=2)
    n.fc6, n.relu6 = fc_relu(n.pool5, 4096, param=param)
    if train:
        n.drop6 = fc7input = L.Dropout(n.relu6, in_place=True)
    else:
        fc7input = n.relu6
    n.fc7, n.relu7 = fc_relu(fc7input, 4096, param=param)
    if train:
        n.drop7 = fc8input = L.Dropout(n.relu7, in_place=True)
    else:
        fc8input = n.relu7
    # always learn fc8 (param=learned_param)
    fc8 = L.InnerProduct(fc8input, num_output=2622, param=learned_param)
    # give fc8 the name specified by argument `classifier_name`
    fc9_elq = L.InnerProduct(fc8, num_output=1, param=learned_param)
    n.__setattr__(classifier_name, fc9_elq)
    if not train:
        n.probs = L.Softmax(fc9_elq)
    if label is not None:
        n.label = label
        n.loss = L.SoftmaxWithLoss(fc9_elq, n.label)
        n.acc = L.Accuracy(fc9_elq, n.label)
    # write the net to a temporary file and return its filename
    with tempfile.NamedTemporaryFile(delete=False) as f:
        f.write(str(n.to_proto()))
        return f.name 

In [ ]:
#Fuera de uso###################

from caffe import layers as L
from caffe import params as P

def primer_net(train=True, learn_all=False, subset=None):
    data_path = '/home/gonzalo/data/MIT/'
    model = '/home/gonzalo/dev/vgg_face_caffe/VGG_FACE_train_val.prototxt' #'/home/gonzalo/caffe/models/bvlc_googlenet/train_val.prototxt'# 
    weights = '/home/gonzalo/dev/vgg_face_caffe/VGG_FACE.caffemodel'
    if subset is None:
        subset = 'train' if train else 'test'
    source = data_path + '%s_0.txt' % subset
    print source
    transform_param = dict(mirror=train, #crop_size=227,
        mean_file= None)
    primer_data, primer_label = L.ImageData(
        transform_param=transform_param, source=source,
        batch_size=20, ntop=2)
    print primer_data, primer_label
    #return vgg_face_net(data=primer_data, label=primer_label, train=True,
    #                num_classes=10,
    #                classifier_name='fc9_elq',
    #                learn_all=False)
    return caffe.Net(model,
                    weights,
                    caffe.TRAIN)


In [ ]:
#print primer_net(train=True, learn_all=False, subset=None)

#Directories definition function
#Directory to build processed dataset
work_path = "/home/gonzalo/data/MIT"#os.path.dirname(os.path.abspath(__file__))
print work_path #for debug purposes
make_sure_path_exists(work_path)
#Directory to from where to extract data